In [1]:
import os
import sys
import numpy as np
from fbs import FBSDataset, FBSConfig
  
from mrcnn import model as modellib    
  
ROOT_DIR = os.path.abspath('../../')
sys.path.append(ROOT_DIR)         
      
LOG_DIR = os.path.join(ROOT_DIR, 'logs')
MODEL_DIR = os.path.join(LOG_DIR, "mask_rcnn")
DATASET_DIR = os.path.join(ROOT_DIR, 'data/')

dataset_val = FBSDataset()        
dataset_val.load_data(DATASET_DIR, subset='validate')
dataset_val.prepare() 

Using TensorFlow backend.
loading data:   0%|          | 0/43 [00:00<?, ?it/s]

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1580837802784881484
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10257639015
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17922900125706554562
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


loading data: 100%|██████████| 43/43 [00:47<00:00,  1.11s/it]


In [2]:
class InferenceConfig(FBSConfig):
    IMAGES_PER_GPU = 1
inference_config = InferenceConfig()

model = modellib.MaskRCNN(        
         mode='inference',
         config=inference_config,
         model_dir=MODEL_DIR
         )      
  
model_path = '../../logs/mask_rcnn/fbs_resnet50_da_tf_nominimask_20190225T1120/mask_rcnn_fbs_resnet50_da_tf_nominimask__0030.h5'
model.load_weights(model_path, by_name=True)

Re-starting from epoch 30


In [12]:
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

plt.gray()

def dice_coef(y_true, y_pred):     
    intersection = np.sum(y_pred[y_true==1])
    insum = np.sum(y_true) + np.sum(y_pred)
    
    if insum == 0:
        return 1
    
    return 2. * intersection / insum

<Figure size 432x288 with 0 Axes>

In [13]:
f1_scores = []
iou_scores = []
dice_scores = []


for img_id in dataset_val.image_ids:
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        dataset_val, inference_config, img_id, use_mini_mask=False)
    
    results = model.detect([image], verbose = 0)
    r = results[0]
    pred = r['masks']

    
    if(pred.shape[2] == 0):
        pred = np.zeros((256,256))

    if(mask.shape[2] == 0):
        mask = np.zeros((256,256))

    mask = np.asarray(np.squeeze(mask), dtype=np.bool)
    pred = np.squeeze(pred)
    

    f1_scores.append(f1_score(mask.flatten(), pred.flatten()))
    iou_scores.append(jaccard_similarity_score(mask.flatten(), pred.flatten()))
    dice_scores.append(dice_coef(mask.flatten(), pred.flatten()))
    
    
    
print(np.mean(f1_scores))
print(np.mean(iou_scores))
print(np.mean(dice_scores))

0.6852639240582237
0.9933114436778878
0.7810658616255002


In [31]:
from mrcnn.utils import extract_bboxes, non_max_suppression
def ensemble_prediction(model, config, image):

    """ Test time augmentation method using non-maximum supression"""

    masks = []
    scores = []
    boxes = []
    

    results = {}

    result = model.detect([image], verbose=0)[0]
    
    if result['masks'].shape[2] == 0:
        return result
    
    masks.append(result['masks'])
    scores.append(result['scores'])
    boxes.append(extract_bboxes(result['masks']))

    temp_img = np.fliplr(image)
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.fliplr(result['masks'])
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(extract_bboxes(mask))

    temp_img = np.flipud(image)
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.flipud(result['masks'])
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(extract_bboxes(mask))

    angle = np.random.choice([1, -1])
    temp_img = np.rot90(image, k=angle, axes=(0, 1))
    result = model.detect([temp_img], verbose=0)[0]
    mask = np.rot90(result['masks'], k=-angle, axes=(0, 1))
    masks.append(mask)
    scores.append(result['scores'])
    boxes.append(extract_bboxes(mask))

    masks = np.concatenate(masks, axis=-1)
    scores = np.concatenate(scores, axis=-1)
    boxes = np.concatenate(boxes, axis=0)

    # config.DETECTION_NMS_THRESHOLD)
    keep_ind = non_max_suppression(boxes, scores, 0.1)[0]
    masks = masks[:, :, keep_ind]
    scores = scores[keep_ind]

    results['masks'] = masks
    results['scores'] = scores

    return results

In [34]:
f1_scores = []
iou_scores = []
dice_scores = []


for img_id in dataset_val.image_ids:
    image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        dataset_val, inference_config, img_id, use_mini_mask=False)
    
    results = ensemble_prediction(model, inference_config, image)
    r = results
    pred = r['masks']
    
    if(len(pred.shape) > 2 and pred.shape[2] == 0):
        pred = np.zeros((256,256))

    if(mask.shape[2] == 0):
        mask = np.zeros((256,256))

    mask = np.asarray(np.squeeze(mask), dtype=np.bool)
    pred = np.squeeze(pred)
    

    f1_scores.append(f1_score(mask.flatten(), pred.flatten()))
    iou_scores.append(jaccard_similarity_score(mask.flatten(), pred.flatten()))
    dice_scores.append(dice_coef(mask.flatten(), pred.flatten()))
    
    
    
print(np.mean(f1_scores))
print(np.mean(iou_scores))
print(np.mean(dice_scores))

0.6856554314972212
0.9932094610930773
0.7814573690644978
